# E-flux* functions used to reconstruct context specific GSMMs

#### To use the Eflux function you need:

1. The model in an Excel spreadsheet with the 'RXNS' sheet specified. This can easily be created with the RAVEN toolbox in MATLAB



2. A tab seperated TPM expression matrix with very specific column headers. Column one should be 'Gene' and the gene id's should match the genes in the model. The other columns should represent the experimental condition samples and could be more than one replicate as long as the experimental condition regular expression is in each header. For example, if you have condition1_1 and condition1_2 as column headers and 'condition1' is specified in conditions then it will use both the latter columns to calculate a mean expression

3. A conditions list that specified all the experimental conditions that you wish to calculate flux constraints for and are headers in the TPM expression matrix

### PYTHON MODULES AND LOCAL FUNCTIONS

In [1]:
import os
import sys
import csv
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from statistics import stdev,mean
from math import sqrt,log

In [2]:
def getModelEquations(model):
    dfRxns = pd.read_excel(model,sheet_name='RXNS')
    equations = dict(zip(dfRxns['ID'],dfRxns['EQUATION']))    
    return equations

In [3]:
def AorBMean(geneAssoc,means):
    m = 0
    for gene in geneAssoc:
            if gene in means:
                m += means[gene]
    return m

In [4]:
def setFluxConstraints(model,tpm,condition):
    
    # filter expression matrix
    tpm = pd.read_csv(tpm,sep='\t',index_col='Gene')
    tpm = tpm.filter(like=condition)
    means = dict(tpm.mean(axis=1))

    # set bounds
    model = pd.read_excel(model,sheet_name='RXNS',index_col='ID')
    bounds = {}
    for index,row in model.iterrows():
        if pd.isna(model.loc[index,'GENE ASSOCIATION']) == True:
            bounds[index] = 1000
        else:
            geneAssoc = row['GENE ASSOCIATION'].split(' or ')
            bound = AorBMean(geneAssoc,means)
            if bound == 0:
                bounds[index] = 1000
            elif bound > 0 and bound <= 1:
                bounds[index] = bound
            else:
                bounds[index] = log(bound)
    return bounds

In [5]:
def setReactionBoundaries(bounds,equations):
    reactionBounds = defaultdict(dict)
    for reaction in bounds:
        if '<=>' in equations[reaction]:
            reactionBounds[reaction]['LB'] = round(-bounds[reaction],10)
            reactionBounds[reaction]['UB'] = round(bounds[reaction],10)
        else:
            reactionBounds[reaction]['LB'] = 0
            reactionBounds[reaction]['UB'] = round(bounds[reaction],10)
    return reactionBounds

### DEFINE THE PARAMETERS

In [10]:
# this is the specific experimental conditions that we want to extract TPM values and create 
# context-specific models for. This can be changed as you wish. 

# the TPMMatrix

tpm = 'data/tpm_mat.csv'

conditions = ['nonDEN_Liver_CD','nonDEN_Liver_WD','DEN_Tumour_WD','DEN_AdjLiver_WD','DEN_Liver_CD']

# the generic model we want to impose flux boundaries on. This model needs to be in Excel format that
# can be constructed using the exportToExcelFormat function in the Raven2.0 toolbox
model = str('model/MMRNHep/WD/MMRNHep-WD.xlsx')

### GET THE FLUX BOUNDARIES AND WRITE TO FILE

In [11]:
dfRxns = pd.read_excel(model,sheet_name='RXNS',index_col='ID')
for condition in conditions:
    print(condition)
    
    # get the flux constraints
    fluxConstraints = setFluxConstraints(model,tpm,condition)
    boundaries = setReactionBoundaries(fluxConstraints,getModelEquations(model))
    
    # write this to a file that can be used to impose boundaries on the model
    fout = open('data/Eflux/' + condition + '.csv','w')
    fout.write('rxn\tLB\tUB\n')
    for rxn in boundaries:
        if dfRxns.at[rxn,'SUBSYSTEM'] != 'Exchange reactions':
            fout.write(rxn + '\t')
            fout.write(str(boundaries[rxn]['LB']) + '\t')
            fout.write(str(boundaries[rxn]['UB']) + '\n')
        else:
            pass
    fout.close()

nonDEN_Liver_CD
nonDEN_Liver_WD
DEN_Tumour_WD
DEN_AdjLiver_WD
DEN_Liver_CD
